In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from datasets import Dataset
import pandas as pd
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
from sklearn.model_selection import train_test_split
from trl import SFTTrainer
import bitsandbytes as bnb
from safetensors.torch import load_file


In [2]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True, device_map="auto")
model = prepare_model_for_kbit_training(model)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
df = pd.read_excel("train.xlsx")
df = df.dropna()
train_df, test_df = train_test_split(df, test_size=0.05, random_state=42)
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [4]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=32,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    target_modules=[
        "self_attn.o_proj",
        "self_attn.qkv_proj",
        "mlp.gate_up_proj",
        "mlp.down_proj"
    ],
)


In [5]:
model = get_peft_model(model, peft_config)
tokenizer.pad_token = tokenizer.eos_token

In [6]:
def format_dataset(examples):
    prompts = []
    for text, gloss in zip(examples['English Text'], examples['BSL Gloss']):
        prompt = f"""### SYSTEM: Translate the input text to British Sign Language (BSL) gloss accurately.

### INPUT: {text}

### OUTPUT:
{gloss}"""
        prompts.append(prompt)
    
    # Tokenize inputs
    model_inputs = tokenizer(
        prompts,
        max_length=100,
        padding='max_length',
        truncation=True,
        return_tensors=None
    )
    
    # Set labels
    model_inputs['labels'] = model_inputs['input_ids'].copy()
    
    return model_inputs

In [7]:
train_dataset = train_dataset.map(
    format_dataset,
    batched=True,
    remove_columns=train_dataset.column_names,
    desc="Processing train dataset"
)

test_dataset = test_dataset.map(
    format_dataset,
    batched=True,
    remove_columns=test_dataset.column_names,
    desc="Processing test dataset"
)

Processing train dataset:   0%|          | 0/450 [00:00<?, ? examples/s]

Processing test dataset:   0%|          | 0/24 [00:00<?, ? examples/s]

In [8]:
training_args = TrainingArguments(
    output_dir="./exp_llama",
    num_train_epochs=10,
    per_device_train_batch_size=1,   
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,  
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_dir="./Advance_llama",
    logging_steps=5,
    evaluation_strategy="steps",
    eval_steps=20,
    save_strategy="steps",
    save_steps=20,
    load_best_model_at_end=True,
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    max_grad_norm=0.3,
    group_by_length=True,
    remove_unused_columns=False,
    fp16=True,
    # Added these arguments
    gradient_checkpointing=True,
    optim="paged_adamw_32bit"
)

C:\Users\subbu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    packing=False,
    dataset_text_field="input_ids",
)


C:\Users\subbu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
C:\Users\subbu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\trl\trainer\sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
C:\Users\subbu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\trl\trainer\sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you 

In [10]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: subbu27498 (subbu27498-signapse). Use `wandb login --relogin` to force relogin


  0%|          | 0/280 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
C:\Users\subbu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\transformers\models\llama\modeling_llama.py:655: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
C:\Users\subbu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\bitsandbytes\nn\modules.py:435: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
C:\Users\subbu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local

{'loss': 3.9522, 'grad_norm': 0.9316175580024719, 'learning_rate': 1.785714285714286e-05, 'epoch': 0.18}
{'loss': 3.8352, 'grad_norm': 1.037092685699463, 'learning_rate': 3.571428571428572e-05, 'epoch': 0.36}
{'loss': 3.568, 'grad_norm': 1.3796314001083374, 'learning_rate': 5.3571428571428575e-05, 'epoch': 0.53}
{'loss': 3.052, 'grad_norm': 1.2398269176483154, 'learning_rate': 7.142857142857143e-05, 'epoch': 0.71}


  0%|          | 0/24 [00:00<?, ?it/s]

...

{'loss': 0.8167, 'grad_norm': 1.2580642700195312, 'learning_rate': 1.130009458668863e-05, 'epoch': 8.0}
{'loss': 0.8262, 'grad_norm': 1.3295472860336304, 'learning_rate': 9.403099714207175e-06, 'epoch': 8.18}
{'loss': 0.7745, 'grad_norm': 1.2822660207748413, 'learning_rate': 7.663790038585793e-06, 'epoch': 8.36}
{'loss': 0.9262, 'grad_norm': 1.2643542289733887, 'learning_rate': 6.088921331488568e-06, 'epoch': 8.53}


  0%|          | 0/24 [00:00<?, ?it/s]

{'eval_loss': 1.4339872598648071, 'eval_runtime': 8.1696, 'eval_samples_per_second': 2.938, 'eval_steps_per_second': 2.938, 'epoch': 8.53}


C:\Users\subbu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torch\utils\checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


KeyboardInterrupt: 

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig
from peft import PeftModel, PeftConfig
import torch

checkpoint_path = "exp_llama/checkpoint-180"

tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16
)

model_name = "meta-llama/Llama-3.1-8B-Instruct"
config = AutoConfig.from_pretrained(model_name)


model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    config=config,
    quantization_config=bnb_config 
)
model = PeftModel.from_pretrained(model, checkpoint_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Model successfully loaded with 4-bit precision and LoRA adapter.")



Unused kwargs: ['bnb_8bit_compute_dtype']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model successfully loaded with 4-bit precision and LoRA adapter.


In [2]:
# Example inference
input_text = """### SYSTEM: Translate the input text to British Sign Language (BSL) gloss. 
### INPUT: We provide digital sign language displays for clear, concise, and easy-to-understand visual information.


### OUTPUT: 

"""

inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to(device)

# Generate outputs
outputs = model.generate(
    inputs_embeds=inputs,
    num_return_sequences=1,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    do_sample=True,
    temperature=0.7,
    top_p=0.95
)

res = tokenizer.decode(outputs[0], skip_special_tokens=True)
res

AttributeError: 

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig
from peft import PeftModel, PeftConfig
import torch
from safetensors.torch import load_file

# Load model and tokenizer
checkpoint_path = r"c:\Users\subbu\Desktop\Llama-3.1-train\results\checkpoint-540"
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16
)

model_name = "meta-llama/Llama-3.1-8B-Instruct"
config = AutoConfig.from_pretrained(model_name)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    config=config,
    quantization_config=bnb_config 
)
model.resize_tokens_embeddings(len(tokenizer))
model = PeftModel.from_pretrained(model, checkpoint_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load soft prompts from checkpoint
soft_prompt_path = "./bsl_soft_prompt_model/checkpoint-145"
try:
    state_dict = load_file(f"{soft_prompt_path}/adapter_model.safetensors")
    soft_prompt_key = [k for k in state_dict.keys() if 'prompt_embeddings' in k][0]
    soft_prompt = state_dict[soft_prompt_key]
except:
    try:
        state_dict = torch.load(f"{soft_prompt_path}/rng_state.pth")
        soft_prompt_key = [k for k in state_dict.keys() if 'prompt_embeddings' in k][0]
        soft_prompt = state_dict[soft_prompt_key]
    except:
        raise ValueError("Could not load soft prompts from checkpoint")

# Ensure proper shape and device
if len(soft_prompt.shape) == 2:
    soft_prompt = soft_prompt.unsqueeze(0)  # [1, 50, 4096]
soft_prompt = soft_prompt.to(device).to(torch.float16)
print(f"Loaded soft prompt with shape: {soft_prompt.shape}")

def generate_with_soft_prompt(model, tokenizer, input_text, soft_prompt, max_length=75):
    # Tokenize input
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    # Get input embeddings
    input_embeds = model.get_input_embeddings()(inputs['input_ids'])
    
    # Expand soft prompt to match batch size
    batch_size = input_embeds.shape[0]
    soft_prompt_expanded = soft_prompt.expand(batch_size, -1, -1)
    
    # Concatenate soft prompt with input embeddings
    inputs_embeds = torch.cat([soft_prompt_expanded, input_embeds], dim=1)
    
    # Adjust attention mask
    soft_prompt_mask = torch.ones(
        (batch_size, soft_prompt.shape[1]),
        device=inputs['attention_mask'].device,
        dtype=inputs['attention_mask'].dtype
    )
    attention_mask = torch.cat([soft_prompt_mask, inputs['attention_mask']], dim=1)
    
    # Generate with embeddings
    outputs = model.generate(
        inputs_embeds=inputs_embeds,
        attention_mask=attention_mask,
        max_new_tokens=max_length,
        num_return_sequences=1,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.7,
        top_p=0.95
    )
    
    # Decode and return
    return tokenizer.decode(outputs[0], skip_special_tokens=True)



Unused kwargs: ['bnb_8bit_compute_dtype']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

AttributeError: 'LlamaForCausalLM' object has no attribute 'resize_tokens_embeddings'

In [2]:
from tqdm.auto import tqdm 
import pandas as pd
def process_csv(input_file, output_file, text_column='Text'):
    """
    Process a CSV file and add BSL gloss translations
    
    Args:
        input_file (str): Path to input CSV file
        output_file (str): Path to save output CSV file
        text_column (str): Name of the column containing English text
    """
    # Read the CSV file
    df = pd.read_excel(input_file)
    
    # Create a new column for translations
    df['Translated_glosses'] = ''
    
    # Process each row
    for idx in tqdm(range(len(df)), desc="Translating"):
        text = df.iloc[idx][text_column]
        
        # Skip empty or invalid entries
        if pd.isna(text) or not isinstance(text, str):
            continue
            
        # Prepare input text
        input_text = f"""### SYSTEM: Translate the input text to British Sign Language (BSL) gloss. 

### INPUT: {text}

### OUTPUT: 
"""
        
        # Generate translation
        result = generate_with_soft_prompt(model, tokenizer, input_text, soft_prompt)
        
        # Extract translated text
        try:
            translated_text = result.split("OUTPUT:")[-1].strip()
            print(result)
            df.at[idx, 'Translated_glosses'] = translated_text
        except:
            print(f"Error processing row {idx}")
            df.at[idx, 'Translated_glosses'] = ''
            
        # Save progress periodically
        if idx % 100 == 0:
            df.to_csv(output_file, index=False)
            
    # Save final result
    df.to_csv(output_file, index=False)
    print(f"Translation completed. Results saved to {output_file}")

# Example usage
input_file = "test.xlsx"
output_file = "test_with_translations.csv"
process_csv(input_file, output_file)

Translating:   0%|          | 0/99 [00:00<?, ?it/s]

WELCOME SIGNAPSE THIS SIGNAPSE
GENERATIVE AI SIGNING TRANSLATION SOFTWARE CREATE SIGNING CREATE SIGNING
SIGNING NEED VIDEO SIGNING HOW CREATE VIDEO SIGNING VIDEO SIGNING USE OUR AI TECHNOLOGY CREATE VIDEO SIGNING VIDEO HAVE CONTENT POINT-LEFT POINT-RIGHT POINT-RIGHT VIDEO SIGNING VIDEO SIGNING VIDEO HAVE VIDEO SIGNING VIDEO POINT-LEFT POINT-RIGHT POINT-RIGHT HAVE VIDEO SIGNING VIDEO POINT-LEFT POINT-RIGHT POINT-RIGHT HAVE VIDEO SIGNING VIDEO
SIGNAPSE USE BIG VAST COLLECTION SIGNING VIDEO CREATE SIGNING PERSON TRANSLATOR WHAT SIGNAPSE SIGNING ACCURATE POINT-THIS POINT-THIS POINT-THIS POINT-THIS POINT-THIS POINT-THIS POINT-THIS POINT-THIS POINT-THIS POINT-THIS POINT-THIS POINT-THIS POINT-THIS POINT-THIS POINT-THIS POINT-THIS
CLIENT HAVE TRUST WE DO OUR WORK ALSO TEAM HAVE HIGH AIM-RANGE AIM-RANGE AIM-RANGE AIM-RANGE AIM-RANGE AIM-RANGE AIM-RANGE AIM-RANGE AIM-RANGE AIM-RANGE AIM-RANGE AIM-RANGE AIM-RANGE AIM-RANGE AIM-RANGE AIM-RANGE AIM-RANGE AIM-RANGE AIM-RANGE AIM-RANGE AIM-RANGE
OUR 

In [10]:
# Example usage
input_text = """### SYSTEM: Translate the input text to British Sign Language (BSL) gloss. 

### INPUT: Our AI technology chooses a combination of sign language videos based on the content provided.

### OUTPUT: 
"""

# Generate with soft prompts
result = generate_with_soft_prompt(model, tokenizer, input_text, soft_prompt)
print("\nGenerated BSL gloss:")
output_text = result.split("OUTPUT:")[-1].strip()
print(output_text)


Generated BSL gloss:
ALSO AI TECHNOLOGY MATCH SIGNING VIDEO WHICH MATERIAL INPUT CONTENT-MATERIAL
